In [17]:
!pip install -q --upgrade torch transformers datasets accelerate evaluate

In [18]:
import torch
print("GPU available:", torch.cuda.is_available())


GPU available: True


**Loading Dataset**

In [19]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [20]:
print(dataset["train"][0])



{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

Tokenizer and model
Used DistilBIRT

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
).to("cuda")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Tokenizing Dataset**

In [22]:
def tokenize_function(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

**Training and Evaluation set**

In [23]:
train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(3000))
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))


**Training Configaration**

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    load_best_model_at_end=True,
    report_to="none"
)


Defining Metrices

In [29]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


Training Model



In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-1641198628.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.333800,0.320346,0.866000
2,0.216600,0.421726,0.862000


TrainOutput(global_step=750, training_loss=0.30674410120646156, metrics={'train_runtime': 173.7645, 'train_samples_per_second': 34.53, 'train_steps_per_second': 4.316, 'total_flos': 397402195968000.0, 'train_loss': 0.30674410120646156, 'epoch': 2.0})

**Evaluation**

In [31]:
trainer.evaluate()


{'eval_loss': 0.32034584879875183,
 'eval_accuracy': 0.866,
 'eval_runtime': 7.9272,
 'eval_samples_per_second': 126.147,
 'eval_steps_per_second': 15.768,
 'epoch': 2.0}

**Test on Custom Text**

In [32]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to("cuda")
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(dim=-1).item()
    return "Positive" if prediction == 1 else "Negative"

texts = [
    "The movie was absolutely fantastic!",
    "This was a boring and poorly written film."
]

for t in texts:
    print(t, "→", predict_sentiment(t))


The movie was absolutely fantastic! → Positive
This was a boring and poorly written film. → Negative
